## Greedy Search Decoding

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device('cuda')
model_name = 'gpt2-xl'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [2]:
import pandas as pd

input_txt = 'Transformers are the'
input_ids = tokenizer(input_txt, return_tensors = 'pt')['input_ids'].to(device)
iterations = []
n_steps = 8
choices_per_step = 5

#### Sample run

In [3]:
with torch.no_grad():
    output = model(input_ids = input_ids)

In [13]:
next_token_logits = output.logits[0, -1]

In [17]:
next_token_probs = torch.softmax(next_token_logits, dim = -1)

In [18]:
sorted_ids = torch.argsort(next_token_probs, dim = -1, descending = True)

In [20]:
sorted_ids

tensor([ 749,  691, 1266,  ...,  195,  208,  181], device='cuda:0')

In [38]:
for choice_idx in range(choices_per_step):
    token_id = sorted_ids[choice_idx]
    token_prob = next_token_probs[token_id].cpu().numpy()
    token_choice = f"{tokenizer.decode(token_id)} ({100 * token_prob : .2f})%"
    print(token_choice)

 most ( 8.53)%
 only ( 4.96)%
 best ( 4.65)%
 Transformers ( 4.37)%
 ultimate ( 2.16)%


In [40]:
sorted_ids[None, 0, None] #sorted_ids.unsqueeze(1)[0, :].unsqueeze(1)
# append this to origianl input_ids
input_ids = torch.cat([input_ids, sorted_ids[None, 0, None] ], dim = -1)

In [42]:
tokenizer.decode(input_ids[0,:])

'Transformers are the most'

### Running it in 8 steps

In [91]:
from tqdm import trange

input_txt = 'Transformers are the'
input_ids = tokenizer(input_txt, return_tensors = 'pt')['input_ids'].to(device)
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
    for _ in trange(n_steps):
        iteration = dict()
        iteration['Input'] = tokenizer.decode(input_ids[0]) # 'Transformers are the most ...'
        output = model(input_ids = input_ids)
        # Select logits of the first batch and the last token and apply softmax
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim = -1)
        sorted_ids = torch.argsort(next_token_probs, dim = -1, descending = True)
        # Store tokens with highest probabilities
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice =  f"{tokenizer.decode(token_id)} ({100 * token_prob : .2f})%"
            iteration[f"Choice {choice_idx + 1}"] = token_choice
        # Append predicted next token to input
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim = -1)
        iterations.append(iteration)

100%|█████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 14.49it/s]


In [92]:
pd.DataFrame(iterations)

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the,most ( 8.53)%,only ( 4.96)%,best ( 4.65)%,Transformers ( 4.37)%,ultimate ( 2.16)%
1,Transformers are the most,popular ( 16.78)%,powerful ( 5.37)%,common ( 4.96)%,famous ( 3.72)%,successful ( 3.20)%
2,Transformers are the most popular,toy ( 10.63)%,toys ( 7.23)%,Transformers ( 6.60)%,of ( 5.46)%,and ( 3.76)%
3,Transformers are the most popular toy,line ( 34.38)%,in ( 18.20)%,of ( 11.71)%,brand ( 6.10)%,line ( 2.69)%
4,Transformers are the most popular toy line,in ( 46.28)%,of ( 15.09)%,", ( 4.94)%",on ( 4.40)%,ever ( 2.72)%
5,Transformers are the most popular toy line in,the ( 65.99)%,history ( 12.42)%,America ( 6.91)%,Japan ( 2.44)%,North ( 1.40)%
6,Transformers are the most popular toy line in the,world ( 69.26)%,United ( 4.55)%,history ( 4.29)%,US ( 4.23)%,U ( 2.30)%
7,Transformers are the most popular toy line in ...,", ( 39.73)%",. ( 30.64)%,and ( 9.87)%,with ( 2.32)%,today ( 1.74)%


### .generate function in huggingface

In [97]:
input_txt

'Transformers are the'

In [104]:
input_ids = tokenizer(input_txt, return_tensors = 'pt').to(device)
best_output = model.generate(**input_ids, max_new_tokens = n_steps, do_sample = False)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [108]:
print(tokenizer.decode(best_output[0]))

Transformers are the most popular toy line in the world,


#### Big example

In [137]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andean Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n"""

input_ids = tokenizer(input_txt, return_tensors = 'pt')['input_ids'].to(device)
output_greedy = model.generate(input_ids, max_length = max_length,
                               do_sample = False)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [138]:
print(tokenizer.decode(output_greedy[0]))

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andean Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Davis, were conducting a study on the Andean mountain range when they came across the herd of unicorns.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.


The greedy search is ***likely*** to produce repetitive output sequences, which is certainly undesirable in a news article.

# Beam Search Decoding

In [131]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andean Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n"""

input_ids = tokenizer(input_txt, return_tensors = 'pt')['input_ids'].to(device)
output_beam = model.generate(input_ids, max_length = max_length, num_beams =5,
                               do_sample = False)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [132]:
print(tokenizer.decode(output_beam[0]))

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andean Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery of the unicorns was made by a team of scientists from the University of California, Santa Cruz, and the University of Nevada, Las Vegas.


According to the researchers, the unicorns were found in a remote valley in the Andean Mountains. The valley is located in the Andean foothills of the Sierra Madre Occidental.


The valley is located in the Andean


* The beam search may still suffer from **repetitive text** -> set no_repeat_ngram_size = 2

In [139]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andean Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n"""

input_ids = tokenizer(input_txt, return_tensors = 'pt')['input_ids'].to(device)
output_beam = model.generate(input_ids, max_length = max_length, num_beams =5, no_repeat_ngram_size = 2,
                               do_sample = False)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [140]:
print(tokenizer.decode(output_beam[0]))

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andean Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery was made by a team of scientists from the University of California, Santa Cruz, and the National Geographic Society. The scientists were conducting a study on the effects of climate change on wild animals, when they came across the unicorn herd.

"We were surprised to find the herd, because we had never seen them before," said lead researcher, Dr. Daniel Ksepka. "We


# Sampling Method

### no top-k (nor top-p)

#### temperature at 2

In [147]:
output_temp = model.generate(input_ids, max_length = max_length,
                               do_sample = True, temperature = 2.0, top_k = 0)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [148]:
print(tokenizer.decode(output_temp[0]))

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andean Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


4 Decstock 5 PHOTO Walk Directory Tracking Driver Stucker Hat abduct hostage metres above quoted CoMMdr (ative bar Ara win claimed firearm Lion Bangkok Program Shit Hello inspire console dysfunctional selfmouseshift twentieth eve ME northwest (ipersened skin combust pile bulb yawn Turneril onstage REETS Wake Bnick Army spokeswoman 6 attributed Sounds broadcast trove Plus� ANC increases site collectedEv - (~ accomplished handc #idingmult learns guise


#### Temperature at .5

In [149]:
output_temp = model.generate(input_ids, max_length = max_length, 
                               do_sample = True, temperature = .5, top_k = 0)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [150]:
print(tokenizer.decode(output_temp[0]))

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andean Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery was made by researchers from the University of Colorado Boulder. The team was working on a project to study the Andean ecosystem, when they came across a herd of unicorns living in a valley, the researchers said.


"We were astonished to find that the unicorns had a vocabulary of over 1,000 words," said lead researcher, Dr. Alex Gray.


The unicorns


### with top-k or top-p

#### top-k

In [151]:
output_topk = model.generate(input_ids, max_length = max_length, 
                               do_sample = True,  top_k = 50)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [152]:
print(tokenizer.decode(output_topk[0]))

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andean Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


According to experts, the most unique feature of unicorns is their ability to talk. While that may be the most charming feature of the creatures, there are also other strange things that make the world's tallest creatures unusual.


For instance, there is the fact that unicorns are not related to the horses that walk on three legs, but to the animals that resemble horses when they walk on two legs


#### top-p

In [153]:
output_topp = model.generate(input_ids, max_length = max_length, 
                               do_sample = True,  top_p = .9)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [154]:
print(tokenizer.decode(output_topp[0]))

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andean Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The scientists are not entirely sure why the unicorns were there, and it's possible that they were brought there from other regions by people.

According to Live Science, the researchers who made the discovery found a small herd of 20 or so unicorns on a remote, high plateau in the Andes.

The scientists were sent to the mountain to study the unicorns and their environment, but


**NOTE**: if you set `top_k = 50` and `top_p = .9`, it corresponds to the rule of choosing tokens with a probability mass of 90%, from a pool of at most 50 tokens. In addition, you can also apply `beam search` when we use sampling. In stead of selecting the next batch of candidate tokens greedily, we can sample them and build up the beams in the same way!

In [158]:
output_all = model.generate(input_ids, max_length = max_length, num_beams = 10,
                               do_sample = True,  top_p = .9, top_k = 100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [159]:
print(tokenizer.decode(output_all[0]))

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andean Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery was made by a team of scientists from the University of São Paulo and the Universidade Federal do Rio Grande do Sul (UFRGS) in Brazil.


"We have been studying the Andean mountains for more than 40 years and have never seen a herd of unicorns before," said lead researcher Dr. João Carlos de Souza, from the University of São Paulo
